In [90]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [87]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/kortel/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [10]:
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [12]:
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [13]:
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network</div>

In [14]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network'

In [15]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.'

### Featured Images

In [16]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [17]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [18]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [19]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [20]:
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

### Mars Facts

In [21]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.head()

,0,1,2
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"


In [22]:
df.columns=['Description', 'Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [23]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [113]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [114]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.
html = browser.html
#dir(browser)
#link_elems = browser.links.find_by_partial_text('Hemisphere')
#print(dir(link_elems[0]))
#for link in link_elems:
    #print(dir(link))
#print(link_elems)
#for link in link_elems:
    #print("clicking")
    #print(link.text)
    #hemispheres = {}
    #link.click()
    #sample_link = browser.links.find_by_text('Sample')
    #hemispheres['img_link'] = sample_link
    #hemisphere_image_urls.append(hemispheres)
    #browser.back()
#print(hemisphere_image_urls)
main_soup = soup(html, 'html.parser')
a_tags = main_soup.find_all('a', class_='itemLink')
links = [link.get('href') for link in a_tags]
html_links = [link for link in links if 'html' in link]
unique_links =list(set(html_links))
unique_links
#print(unique_links)
#first = unqiue_links[0]

for link in unique_links:
    hemi_info = {}
    full_url = f'https://marshemispheres.com/{link}'
    #print(full_url)
    browser.visit(full_url)
    hemi_soup = soup(browser.html, 'html.parser')
    img_link = hemi_soup.find('a', text='Sample')
    link_text = img_link.get('href')
    hemi_info['img_url'] = f'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/{link_text}'
    title = hemi_soup.find_all('h2')
    hemi_info['title'] = title[0].text
    hemisphere_image_urls.append(hemi_info)

    

[{'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/valles_marineris_enhanced-full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}, {'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/syrtis_major_enhanced-full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'img_url': 'https://data-class-mars-hemispheres.s3.amazonaws.com/Mars_Hemispheres/images/schiaparelli_enhanced-full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}]


In [84]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[]

In [85]:
# 5. Quit the browser
browser.quit()